In [8]:
#pennylane op-t-mize
import pennylane as qml
import pyzx as zx
import tempfile
import random
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumCircuit, ParameterVector
import csv
import time
import japanize_matplotlib # 追加

In [ ]:
#回路セット読み込み
[ds] = qml.data.load("op-t-mize")

pyzx_circuits = []

for i, qscript in enumerate(ds.circuits):
    try:
        # QuantumScript を OpenQASM 2.0 に変換
        qasm_str = qscript.to_openqasm(
            wires=None,
            rotations=False,
            measure_all=False,
            precision=8
        )

        # QASM を一時ファイル経由で PyZX に読み込む
        with tempfile.NamedTemporaryFile(suffix=".qasm", mode='w+', delete=False) as tmpfile:
            tmpfile.write(qasm_str)
            tmpfile.flush()
            zx_circuit = zx.Circuit.load(tmpfile.name)

        # 元の回路名を設定（PyZX Circuit に名前属性を追加）
        zx_circuit.name = ds.circuit_names[i]
        pyzx_circuits.append(zx_circuit)
        
    except Exception as e:
        print(f"回路 {i}（{ds.circuit_names[i]}）の変換に失敗: {e}")

In [4]:
#読み込んだ回路の確認
print(len(pyzx_circuits))
# PyZX の回路を表示

for i, zx_circuit in enumerate(pyzx_circuits):
    dict = zx_circuit.stats_dict()
    print(dict)
    #print(f"gates:{dict['gates']}, qubits:{dict['qubits']}, depth:{dict['depth']},tcount:{dict['tcount']},had:{dict['had']},ohter:{dict['other']},cnot:{dict['cnot']}")


31
{'name': 'mod5_4', 'qubits': 5, 'gates': 63, 'tcount': 28, 'clifford': 35, 'twoqubit': 28, 'cnot': 28, 'had': 6, 'measurement': 0, 'other': 0, 'depth': 0, 'depth_cz': 0}
{'name': 'vbe_adder_3', 'qubits': 10, 'gates': 150, 'tcount': 70, 'clifford': 80, 'twoqubit': 70, 'cnot': 70, 'had': 10, 'measurement': 0, 'other': 0, 'depth': 0, 'depth_cz': 0}
{'name': 'csla_mux_3_original', 'qubits': 15, 'gates': 170, 'tcount': 70, 'clifford': 100, 'twoqubit': 80, 'cnot': 80, 'had': 20, 'measurement': 0, 'other': 0, 'depth': 0, 'depth_cz': 0}
{'name': 'csum_mux_9_corrected', 'qubits': 30, 'gates': 448, 'tcount': 196, 'clifford': 252, 'twoqubit': 168, 'cnot': 168, 'had': 56, 'measurement': 0, 'other': 0, 'depth': 0, 'depth_cz': 0}
{'name': 'qcla_com_7', 'qubits': 24, 'gates': 443, 'tcount': 203, 'clifford': 240, 'twoqubit': 186, 'cnot': 186, 'had': 39, 'measurement': 0, 'other': 0, 'depth': 0, 'depth_cz': 0}
{'name': 'qcla_mod_7', 'qubits': 26, 'gates': 884, 'tcount': 413, 'clifford': 471, 'twoqub

In [ ]:
# 入力の情報をCSVファイルに保存
output_file = "/home/rest/forpyzx/output/0513_op-t-mize.csv"
fieldnames=[
    "Circuit","n_qubits","input_gates","input_t","input_2","input_other","input_depth",
]

with open(output_file, "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i, zx_circuit in enumerate(pyzx_circuits):
        dict = zx_circuit.stats_dict()
        writer.writerow({
            "Circuit": ds.circuit_names[i],
            "n_qubits": dict["qubits"],
            "input_gates": dict["gates"],
            "input_t": dict["tcount"],
            "input_2": dict["twoqubit"],
            "input_other": dict["gates"]-dict["tcount"]-dict["twoqubit"],
            "input_depth": dict["depth"]
        })

In [24]:
qqq = pyzx_circuits[0]
print(qqq)
qqq = zx.basic_optimization(qqq)
qqq = zx.optimize.basic_optimization(qqq)
g = qqq.to_basic_gates().to_graph()
zx.simplify.full_reduce(g)
qqq = zx.extract.extract_circuit(g,up_to_perm=True)
qqq = zx.optimize.basic_optimization(qqq)

Circuit(5 qubits, 0 bits, 63 gates)


In [ ]:
optimize_output_file = "/home/rest/forpyzx/output/0513_op-t-mize_optimize.csv"
fieldnames=[
    "Circuit","optimize_name","n_qubits","optimize_gates","optimize_t","optimize_2","optimize_other","optimize_depth","optimize_depth","time"
]

optimize_list = [
    "original",
    "basic",
    "phase_basic",
    "basic_full_basic",
    "phase_full_basic",
    "basic_clifford_basic",
    "phase_clifford_basic",
    "phase_basic_full_basic",
    "phase_basic_clifford_basic",
]


start_time = time.time()
with open(optimize_output_file, "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i, zx_circuit in enumerate(pyzx_circuits):
        if i >= 28:
            break
        for optimize_name in optimize_list:
            try:
                c = zx_circuit.copy()

                start_time_inner = time.time()

                if optimize_name == "original":
                    a=1
                elif optimize_name == "basic":
                    c = zx.optimize.basic_optimization(c)
                elif optimize_name == "phase_basic":
                    g = c.to_graph()
                    zx.teleport_reduce(g)
                    c = zx.Circuit.from_graph(g)
                    c = zx.optimize.basic_optimization(c)
                elif optimize_name == "basic_full_basic":
                    c = zx.optimize.basic_optimization(c)
                    g = c.to_basic_gates().to_graph()
                    zx.simplify.full_reduce(g)
                    c = zx.extract.extract_circuit(g,up_to_perm=True)
                    c = zx.optimize.basic_optimization(c)
                elif optimize_name == "phase_full_basic":
                    g = c.to_graph()
                    zx.teleport_reduce(g)
                    c = zx.Circuit.from_graph(g)
                    g = c.to_graph()
                    zx.simplify.full_reduce(g)
                    c = zx.extract.extract_circuit(g,up_to_perm=True)
                    c = zx.optimize.basic_optimization(c)
                elif optimize_name == "basic_clifford_basic":
                    c = zx.optimize.basic_optimization(c)
                    g = c.to_graph()
                    zx.simplify.clifford_simp(g)
                    c = zx.extract.extract_circuit(g,up_to_perm=True)
                    c = zx.optimize.basic_optimization(c)
                elif optimize_name == "phase_clifford_basic":
                    g = c.to_graph()
                    zx.teleport_reduce(g)
                    c = zx.Circuit.from_graph(g)
                    g = c.to_graph()
                    zx.simplify.clifford_simp(g)
                    c = zx.extract.extract_circuit(g,up_to_perm=True)
                    c = zx.optimize.basic_optimization(c)

                elif optimize_name == "phase_basic_full_basic":
                    g = c.to_graph()
                    zx.teleport_reduce(g)
                    c = zx.Circuit.from_graph(g)
                    c = zx.optimize.basic_optimization(c)
                    g = c.to_graph()
                    zx.simplify.full_reduce(g)
                    c = zx.extract.extract_circuit(g,   up_to_perm=True)
                    c = zx.optimize.basic_optimization(c)
                elif optimize_name == "phase_basic_clifford_basic":
                    g = c.to_graph()
                    zx.teleport_reduce(g)
                    c = zx.Circuit.from_graph(g)
                    c = zx.optimize.basic_optimization(c)
                    g = c.to_graph()
                    zx.simplify.clifford_simp(g)
                    c = zx.extract.extract_circuit(g,up_to_perm=True)
                    c = zx.optimize.basic_optimization(c)

                end_time_inner = time.time()

                #時間を計測して表示
                print(f"回路 {i}（{ds.circuit_names[i]}）optimizename:{optimize_name},optimized time: {end_time_inner - start_time_inner}秒")


                dict = c.stats_dict()

                #書き込み
                writer.writerow({
                    "Circuit": zx_circuit.name,
                    "optimize_name":   optimize_name,# 最適化の名前をここに記入
                    "n_qubits": dict["qubits"],
                    "optimize_gates":dict["gates"],
                    "optimize_t": dict["tcount"],
                    "optimize_2": dict["twoqubit"],
                    "optimize_other": dict["gates"]-dict["tcount"]-dict["twoqubit"],
                    "optimize_depth": dict["depth"],
                    "time": end_time_inner - start_time_inner
                })

            except Exception as e:
                print(f"optimizename= {optimize_name}回路 {i}（{ds.circuit_names[i]}）の最適化に失敗: {e}")
                continue
        
end_time = time.time()
print(f"処理時間: {end_time - start_time}秒")

回路 0（mod5_4）optimizename:original,optimized time: 9.012222290039062e-05秒
回路 0（mod5_4）optimizename:basic,optimized time: 0.0034999847412109375秒
回路 0（mod5_4）optimizename:phase_basic,optimized time: 0.00863027572631836秒
回路 0（mod5_4）optimizename:basic_full_basic,optimized time: 0.00975489616394043秒
回路 0（mod5_4）optimizename:phase_full_basic,optimized time: 0.012744426727294922秒
回路 0（mod5_4）optimizename:basic_clifford_basic,optimized time: 0.010808467864990234秒
回路 0（mod5_4）optimizename:phase_clifford_basic,optimized time: 0.013365983963012695秒
回路 0（mod5_4）optimizename:phase_basic_full_basic,optimized time: 0.014969110488891602秒
回路 0（mod5_4）optimizename:phase_basic_clifford_basic,optimized time: 0.015560626983642578秒
回路 1（vbe_adder_3）optimizename:original,optimized time: 4.76837158203125e-07秒
回路 1（vbe_adder_3）optimizename:basic,optimized time: 0.0032117366790771484秒
回路 1（vbe_adder_3）optimizename:phase_basic,optimized time: 0.01809978485107422秒
回路 1（vbe_adder_3）optimizename:basic_full_basic,op

In [ ]:
#0522 新抽出の確認
optimize_output_file = "/home/rest/forpyzx/output/0522_op-t-mize_optimize_new_extract.csv"
fieldnames=[
    "Circuit","optimize_name","n_qubits","optimize_gates","optimize_t","optimize_2","optimize_other","optimize_depth","optimize_depth","time"
]

optimize_list = [
    "phase_basic_full_n_basic",
    "phase_basic_clifford_n_basic",
]


start_time = time.time()
with open(optimize_output_file, "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i, zx_circuit in enumerate(pyzx_circuits):
        if i >= 28:
            break
        for optimize_name in optimize_list:
            try:
                c = zx_circuit.copy()

                start_time_inner = time.time()

                if optimize_name == "phase_basic_full_n_basic":
                    g = c.to_graph()
                    zx.teleport_reduce(g)
                    c = zx.Circuit.from_graph(g)
                    c = zx.optimize.basic_optimization(c)
                    g = c.to_graph()
                    zx.simplify.full_reduce(g)
                    c = zx.extract.lookahead_extract(g,   up_to_perm=True)
                    c = zx.optimize.basic_optimization(c)
                elif optimize_name == "phase_basic_clifford_n_basic":
                    g = c.to_graph()
                    zx.teleport_reduce(g)
                    c = zx.Circuit.from_graph(g)
                    c = zx.optimize.basic_optimization(c)
                    g = c.to_graph()
                    zx.simplify.clifford_simp(g)
                    c = zx.extract.lookahead_extract(g,up_to_perm=True)
                    c = zx.optimize.basic_optimization(c)

                end_time_inner = time.time()

                #時間を計測して表示
                print(f"回路 {i}（{ds.circuit_names[i]}）optimizename:{optimize_name},optimized time: {end_time_inner - start_time_inner}秒")


                dict = c.stats_dict()

                #書き込み
                writer.writerow({
                    "Circuit": zx_circuit.name,
                    "optimize_name":   optimize_name,# 最適化の名前をここに記入
                    "n_qubits": dict["qubits"],
                    "optimize_gates":dict["gates"],
                    "optimize_t": dict["tcount"],
                    "optimize_2": dict["twoqubit"],
                    "optimize_other": dict["gates"]-dict["tcount"]-dict["twoqubit"],
                    "optimize_depth": dict["depth"],
                    "time": end_time_inner - start_time_inner
                })

            except Exception as e:
                print(f"optimizename= {optimize_name}回路 {i}（{ds.circuit_names[i]}）の最適化に失敗: {e}")
                continue
        
end_time = time.time()
print(f"処理時間: {end_time - start_time}秒")